In [1]:

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from tqdm.notebook import tqdm
from torch.nn import functional as F
import algos

device = 'cuda'

In [3]:
def conv_block(num_channels):
    return nn.Sequential(
        nn.LazyBatchNorm2d(), nn.ReLU(),
        nn.LazyConv2d(num_channels, 3, 1, 1)
    )

def transition_block(num_channels: int)->nn.Module:
    """
    Used to reduce dimensions 1x1 and halve size AvgPool
    """
    return nn.Sequential(
        nn.LazyBatchNorm2d(), nn.ReLU(),
        nn.LazyConv2d(num_channels, 1),
        nn.AvgPool2d(2, 2)
    )


class DenseBlock(nn.Module):
    def __init__(self, num_convs, num_channels):
        super().__init__()
        layer = []
        for i in range(num_convs):
            layer.append(conv_block(num_channels))
        self.net = nn.Sequential(*layer)

    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # Concat input and output of each block
            # along the channels
            X = torch.cat((X, Y), dim=1)
        return X

In [6]:
class DenseNet(nn.Module):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, 7, 2, 3),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.MaxPool2d(3,2,1)
        )

    def __init__(self, num_channels=64, growth_rate=32, arch=(4,4,4,4), num_classes=10):
        super().__init__()
        self.net = nn.Sequential(self.b1())

        for i, num_convs in enumerate(arch):
            self.net.add_module(f'dense_blk{i+1}', DenseBlock(num_convs, growth_rate))
            # Prev out channels:
            num_channels = num_convs * growth_rate
            if i != len(arch) - 1:
                num_channels //= 2
                self.net.add_module(f'tran_blk{i+1}', transition_block(num_channels))
            self.net.add_module('last', nn.Sequential(
                nn.LazyBatchNorm2d(), nn.ReLU(),
                nn.AdaptiveAvgPool2d((1,1)), nn.Flatten(),
                nn.LazyLinear(num_classes)
            ))
            self.apply(algos.init_cnn)

    def apply_init(self, inputs, init=None):
        self.forward(*inputs)
        if init:
            self.net.apply(init)

    def forward(self, X):
        return self.net(X)

In [13]:
train_loader, _ = algos.load_mnist()
model = algos.fit(DenseNet(), train_loader)

C:\Users\kosti_0b5rpb8\anaconda3\envs\torch_cuda__118\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
C:\Users\kosti_0b5rpb8\anaconda3\envs\torch_cuda__118\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call 

ValueError: expected 4D input (got 2D input)